## データのダウンロード

In [16]:
import requests


def download_file(url, save_path):
    with open(save_path, "wb") as file:
        response = requests.get(url)
        file.write(response.content)


path = "arxiv.jsonl"
download_url = "https://data.together.xyz/redpajama-data-1T/v1.0.0/arxiv/arxiv_023827cd-7ee8-42e6-aa7b-661731f4c70f.jsonl"
download_file(download_url, path)

In [17]:
# データの確認
import json

with open(path, "r") as f:
    d = json.loads(f.readline())

print(d.keys())

dict_keys(['text', 'meta'])


In [20]:
print(d["meta"])

{'content_hash': '91f446bb78faf3093f30e0586b5e78a4', 'timestamp': '', 'source': 'github', 'line_count': 26, 'max_line_length': 78, 'avg_line_length': 28.192307692307693, 'alnum_prop': 0.7421555252387448, 'repo_name': 'coca-cola33/CloudFilm', 'id': '585abba6f2aa05e68e870464ce47fbf0fbf3a35a', 'size': '733', 'binary': False, 'copies': '1', 'ref': 'refs/heads/master', 'path': 'common/src/androidTest/java/kg/jl/common/ExampleInstrumentedTest.java', 'mode': '33188', 'license': 'apache-2.0', 'language': [{'name': 'Java', 'bytes': '887574'}]}


In [13]:
# データの読み込み
import json


with open(path, "r") as f:
    texts = [json.loads(line)["text"] for line in f]

## データの前処理

16767
